<a href="https://colab.research.google.com/github/asyhar22/tour-de-france/blob/main/Tour_de_France.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
# https://www.letour.fr/en/history
# https://www.letour.fr/en/block/history/11821/a3fe7d350ef546e3c27ae83a84fa3a0b --> year of 2021
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=21&type=itg --> individual general 2021
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=21&type=ite --> individual stage 2021
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=21&type=ipg --> points general 2021
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=21&type=ipe --> points stage 2021
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=21&type=img --> climber general 2021 --> check again
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=21&type=ime --> climber stage 2021 --> check again
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=2&type=icg --> combative general 2021 --> check again
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=2&type=ice --> combative stage 2021 --> check again
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=21&type=etg --> team general 2021
# https://www.letour.fr/en/block/history/11821/ranking/7624efa14b30dd23fce5b89100259570?stage=21&type=ete --> team stage 2021
# https://www.letour.fr/en/block/history/11821/stages/43335f5fa657aaa48567acb0d84cf1d1 --> stages 2021
# https://www.letour.fr/en/block/history/11821/jerseys/68233f403db6371888c2121baf02c13c --> jersey wearer 2021
# https://www.letour.fr/en/block/history/11821/winners/78285f82cebf4bd0ee372cb544263c57 --> stages winner 2021

In [3]:
years = list(range(2010,2024))
stage_list = list(range(1,22))

In [ ]:
# url for each year

year_urls = dict()

r = requests.get('https://www.letour.fr/en/history')
soup = BeautifulSoup(r.text,'html.parser')

for year in years:
  path = soup.find('button',{'class': 'dateTabs__link js-tabs','data-tabs-target':year})
  year_path = path['data-tabs-ajax']
  year_urls[year]= f'https://www.letour.fr{year_path}'

year_urls

In [ ]:
# rank url for each year

rank_urls = dict()
for year in years:
  r = requests.get(year_urls[year])
  soup = BeautifulSoup(r.text,'html.parser')

  path = soup.find('button',{'class': 'tabs__link js-tabs-nested','data-tabs-target':'ranking'})
  rank_path = path['data-tabs-ajax']
  rank_urls[year] = f'https://www.letour.fr{rank_path}'

rank_urls

In [6]:
# itg --> individual general (for each year & for each stage)

rank_type = 'itg'
rank_itg = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    rank_itg = pd.concat([rank_itg,to_append], ignore_index=True)

rank_itg.to_csv('Tour de France Individual (General) 2010-2023.csv', index=False)

In [7]:
# ite --> individual stage (for each year & for each stage)

rank_type = 'ite'
rank_ite = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    rank_ite = pd.concat([rank_ite,to_append], ignore_index=True)

rank_ite.to_csv('Tour de France Individual (Stage) 2010-2023.csv', index=False)

In [8]:
# ipg --> points general (for each year & for each stage)

rank_type = 'ipg'
rank_ipg = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    rank_ipg = pd.concat([rank_ipg,to_append], ignore_index=True)

rank_ipg.to_csv('Tour de France Points (General) 2010-2023.csv', index=False)

In [18]:
# ipe --> points stage (for each year & for each stage)

rank_type = 'ipe'
rank_ipe = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    rank_ipe = pd.concat([rank_ipe,to_append], ignore_index=True)

ipe_segment = pd.DataFrame()
for i in rank_ipe.index:
  if len(str(rank_ipe.loc[i,"Rank"])) > 5:
    segment=rank_ipe.loc[i,"Rank"]
  else:
    to_append = rank_ipe.loc[i:i]
    to_append['Segment'] = segment
    ipe_segment = pd.concat([ipe_segment, to_append], ignore_index=True)

ipe_segment.to_csv('Tour de France Points (Stage) 2010-2023.csv', index=False)

In [19]:
# img --> climber general (for each year & for each stage)

rank_type = 'img'
rank_img = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    if len(to_append) < 100:
      rank_img = pd.concat([rank_img,to_append], ignore_index=True)

rank_img.to_csv('Tour de France Climber (General) 2010-2023.csv', index=False)

In [20]:
# ime --> climber stage (for each year & for each stage)

rank_type = 'ime'
rank_ime = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    if len(to_append) < 100:
      rank_ime = pd.concat([rank_ime,to_append], ignore_index=True)

rank_ime.pop(rank_ime.columns[-3])

ime_segment = pd.DataFrame()
for i in rank_ime.index:
  if len(str(rank_ime.loc[i,"Rank"])) > 5:
    segment=rank_ime.loc[i,"Rank"]
  else:
    to_append = rank_ime.loc[i:i]
    to_append['Segment'] = segment
    ime_segment = pd.concat([ime_segment, to_append], ignore_index=True)

ime_segment.to_csv('Tour de France Climber (Stage) 2010-2023.csv', index=False)

In [21]:
# icg --> combative general (for each year & for each stage)

rank_type = 'icg'
rank_icg = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    if len(to_append) < 50:
      rank_icg = pd.concat([rank_icg,to_append], ignore_index=True)

rank_icg.to_csv('Tour de France Combative (General) 2010-2023.csv', index=False)

In [22]:
# ice --> combative stage (for each year & for each stage)

rank_type = 'ice'
rank_ice = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    if len(to_append) < 50:
      rank_ice = pd.concat([rank_ice,to_append], ignore_index=True)

rank_ice.to_csv('Tour de France Combative (Stage) 2010-2023.csv', index=False)

In [23]:
# etg --> team general (for each year & for each stage)

rank_type = 'etg'
rank_etg = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    rank_etg = pd.concat([rank_etg,to_append], ignore_index=True)

rank_etg.to_csv('Tour de France Team (General) 2010-2023.csv', index=False)

In [24]:
# ete --> team stage (for each year & for each stage)

rank_type = 'ete'
rank_ete = pd.DataFrame()
for year in years:
  for stage in stage_list:
    to_append = pd.read_html(f'{rank_urls[year]}?stage={stage}&type={rank_type}', encoding='utf-8')[0]
    to_append['Stages'] = stage
    to_append['Year'] = year
    rank_ete = pd.concat([rank_ete,to_append], ignore_index=True)

rank_ete.to_csv('Tour de France Team (Stage) 2010-2023.csv', index=False)

In [25]:
# stages for each year
stages = pd.DataFrame()
for year in years:
  r = requests.get(year_urls[year])
  soup = BeautifulSoup(r.text,'html.parser')

  path = soup.find('button',{'class': 'tabs__link js-tabs-nested','data-tabs-target':'stages'})
  stages_path = path['data-tabs-ajax']

  to_append = pd.read_html(f'https://www.letour.fr{stages_path}', encoding='utf-8')[0]
  to_append['Year'] = year
  stages = pd.concat([stages,to_append], ignore_index=True)

stages.to_csv('Tour de France Stages 2010-2023.csv', index=False)

In [26]:
# jerseys for each year
jerseys = pd.DataFrame()
for year in years:
  r = requests.get(year_urls[year])
  soup = BeautifulSoup(r.text,'html.parser')

  path = soup.find('button',{'class': 'tabs__link js-tabs-nested','data-tabs-target':'jerseys'})
  jerseys_path = path['data-tabs-ajax']

  to_append = pd.read_html(f'https://www.letour.fr{jerseys_path}', encoding='utf-8')[0]
  to_append['Year'] = year
  jerseys = pd.concat([jerseys,to_append], ignore_index=True)

jerseys.to_csv('Tour de France Jerseys 2010-2023.csv', index=False)

In [27]:
# winners for each year
winners = pd.DataFrame()
for year in years:
  r = requests.get(year_urls[year])
  soup = BeautifulSoup(r.text,'html.parser')

  path = soup.find('button',{'class': 'tabs__link js-tabs-nested','data-tabs-target':'jerseys'})
  winners_path = path['data-tabs-ajax']

  to_append = pd.read_html(f'https://www.letour.fr{winners_path}', encoding='utf-8')[0]
  to_append['Year'] = year
  winners = pd.concat([winners,to_append], ignore_index=True)

winners.to_csv('Tour de France Winners 2010-2023.csv', index=False)